In [33]:
# python函数与tensorflow图

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [34]:
# 把python的函数变为图的目的是为了提高程序的运行速度

In [35]:
# tf.function and auto-graph
# 实现一下elu激活函数
def scaled_elu(z, scale=1., alpha=1.):
    # z >= 0 ? scale * z:scale * alpha * tf.nn.elu
    is_positive = tf.greater_equal(z, 0.)
    # return scale * tf.where(is_positive, z, alpha * tf.nn.elu(z))
    return scale * tf.where(is_positive, z, alpha * (tf.math.exp(z) - 1))


print(scaled_elu(tf.constant(-3.)))
print('-' * 50)
print(scaled_elu(tf.constant([-3. - 2.5])))
print('-' * 50)
# 通过tf.function可以把python的函数变为图实现的函数
scaled_elu_tf = tf.function(scaled_elu)
print(scaled_elu_tf(tf.constant(-3.)))
print('-' * 50)
print(scaled_elu_tf(tf.constant(-5.5)))
print(scaled_elu_tf(tf.constant([-3. - 2.5])))
print('-' * 50)
# 原python函数
print(scaled_elu_tf.python_function is scaled_elu) # True
print(scaled_elu)
print(scaled_elu_tf)  # 执行效率相对较高

tf.Tensor(-0.95021296, shape=(), dtype=float32)
--------------------------------------------------
tf.Tensor([-0.9959132], shape=(1,), dtype=float32)
--------------------------------------------------
tf.Tensor(-0.95021296, shape=(), dtype=float32)
--------------------------------------------------
tf.Tensor(-0.9959132, shape=(), dtype=float32)
tf.Tensor([-0.9959132], shape=(1,), dtype=float32)
--------------------------------------------------
True
<function scaled_elu at 0x000001C4C300FE50>


In [36]:
# # 测试
# %timeit scaled_elu(tf.random.normal((1000, 1000)))
# %timeit scaled_elu_tf(tf.random.normal((1000, 1000)))

In [37]:
# 通过装饰器把python函数变为图
@tf.function
def converge_to_2(n_iters):
    total = tf.constant(0.)
    increment = tf.constant(1.)
    for _ in range(n_iters):
        total += increment
        increment /= 2.
    return total


print(converge_to_2)
print(type(converge_to_2)) # <class 'tensorflow.python.eager.def_function.Function'>
print('-' * 50)
print(converge_to_2(20))

<class 'tensorflow.python.eager.def_function.Function'>
--------------------------------------------------
tf.Tensor(1.9999981, shape=(), dtype=float32)


In [38]:
# 查看tf图的代码
def display_tf_code(func):
    code = tf.autograph.to_code(func)
    from IPython.display import display, Markdown
    display(Markdown(f'```python\n{code}\n```'))

# 传python函数,返回tf图的代码
print(display_tf_code(scaled_elu))

```python
def tf__scaled_elu(z, scale=None, alpha=None):
    with ag__.FunctionScope('scaled_elu', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        is_positive = ag__.converted_call(ag__.ld(tf).greater_equal, (ag__.ld(z), 0.0), None, fscope)
        try:
            do_return = True
            retval_ = ag__.ld(scale) * ag__.converted_call(ag__.ld(tf).where, (ag__.ld(is_positive), ag__.ld(z), ag__.ld(alpha) * (ag__.converted_call(ag__.ld(tf).math.exp, (ag__.ld(z),), None, fscope) - 1)), None, fscope)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)

```

None


In [39]:
# print(display_tf_code(converge_to_2)) # ConversionError
print(type(converge_to_2)) # <class 'tensorflow.python.eager.def_function.Function'>
print(display_tf_code(converge_to_2.python_function))

<class 'tensorflow.python.eager.def_function.Function'>


```python
def tf__converge_to(n_iters):
    with ag__.FunctionScope('converge_to_2', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        total = ag__.converted_call(ag__.ld(tf).constant, (0.0,), None, fscope)
        increment = ag__.converted_call(ag__.ld(tf).constant, (1.0,), None, fscope)

        def get_state():
            return (total, increment)

        def set_state(vars_):
            nonlocal increment, total
            (total, increment) = vars_

        def loop_body(itr):
            nonlocal increment, total
            _ = itr
            total = ag__.ld(total)
            total += increment
            increment = ag__.ld(increment)
            increment /= 2.0
        _ = ag__.Undefined('_')
        ag__.for_stmt(ag__.converted_call(ag__.ld(range), (ag__.ld(n_iters),), None, fscope), None, loop_body, get_state, set_state, ('total', 'increment'), {'iterate_names': '_'})
        try:
            do_return = True
            retval_ = ag__.ld(total)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)

```

None


In [40]:
#tf要把变量定义在函数外面，不能放里边
var = tf.Variable(0.)
@tf.function
def add_10():
     # var = tf.Variable(0.)
    return var.assign_add(10)


print(add_10())

tf.Tensor(10.0, shape=(), dtype=float32)


In [41]:
@tf.function
def cube(z):
    return tf.pow(z, 3)


try:
    print(cube(tf.constant([1.0, 2.0, 3.0])))
except ValueError as ex:
    print(ex)
print('-' * 50)
print(cube(tf.constant([1, 2, 3])))

tf.Tensor([ 1.  8. 27.], shape=(3,), dtype=float32)
--------------------------------------------------
tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


In [42]:
# python泛型设计 通过input_signature加类型限制可以防止调错
@tf.function(input_signature=[tf.TensorSpec([None], tf.int32, name='x')])
def cube(z):
    return tf.pow(z, 3)


try:
    print(cube(tf.constant([1.0, 2.0, 3.0])))
except ValueError as ex:
    print(ex)
print('-' * 50)
print(cube(tf.constant([1, 2, 3])))

Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name='x')).
--------------------------------------------------
tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


In [43]:
cube_func_int32 = cube.get_concrete_function(tf.TensorSpec([None], tf.int32))
print(cube_func_int32)
print('-' * 50)
print(cube)
try:
    print(cube_func_int32(tf.constant([1, 2, 3])))
except Exception as ex:
    print(ex)

ConcreteFunction cube(z)
  Args:
    z: int32 Tensor, shape=(None,)
  Returns:
    int32 Tensor, shape=(None,)
--------------------------------------------------
tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


In [44]:
# print(cube_func_int32 is cube.get_concrete_function())
print(id(cube.get_concrete_function(tf.TensorSpec([None], tf.int32))))
print(id(cube_func_int32)) # 原来函数和新生成的函数一致
print(cube_func_int32 is cube.get_concrete_function(tf.TensorSpec([None], tf.int32)))

1943965814544
1943965814544
True


In [45]:
print(cube_func_int32)
print(cube_func_int32.graph)
print('-' * 50)
# 图的操作
print(cube_func_int32.graph.get_operations())

ConcreteFunction cube(z)
  Args:
    z: int32 Tensor, shape=(None,)
  Returns:
    int32 Tensor, shape=(None,)
FuncGraph(name=cube, id=1943965811280)
--------------------------------------------------
[<tf.Operation 'x' type=Placeholder>, <tf.Operation 'Pow/y' type=Const>, <tf.Operation 'Pow' type=Pow>, <tf.Operation 'Identity' type=Identity>]


In [46]:
pow_op = cube_func_int32.graph.get_operations()[0]
print(pow_op)

name: "x"
op: "Placeholder"
attr {
  key: "_user_specified_name"
  value {
    s: "x"
  }
}
attr {
  key: "dtype"
  value {
    type: DT_INT32
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: -1
      }
    }
  }
}



In [47]:
print(list(pow_op.inputs))
print('-' * 50)
print(list(pow_op.outputs))

[]
--------------------------------------------------
[<tf.Tensor 'x:0' shape=(None,) dtype=int32>]


In [48]:
# Placeholder用来放输入的地方，2.0中不需要，图中依然保留了
print(cube_func_int32.graph.get_operation_by_name("z"))

KeyError: "The name 'z' refers to an Operation not in the graph."

In [50]:
print(cube_func_int32.graph.get_tensor_by_name("z:0"))

KeyError: "The name 'z:0' refers to a Tensor which does not exist. The operation, 'z', does not exist in the graph."

In [ ]:
# 图的信息
print(cube_func_int32.graph.as_graph_def())